### Code

In [1]:
from z3 import *

def z3_to_float(z3_val):
    """
    將已經經過模型求解的 Z3 數值轉換成 float。
    優先使用 as_decimal() 方法，如果返回字串以 '?' 結尾則移除，否則直接轉換。
    """
    try:
        dec_str = z3_val.as_decimal(10)
        if dec_str.endswith('?'):
            dec_str = dec_str[:-1]
        return float(dec_str)
    except Exception:
        return float(str(z3_val))

def calculate_securities_tax_item(tax_item, tp, ep=None, sc=None):
    """
    計算證券交易稅中的某一項目。

    參數：
      - tax_item: 必須為下列其中之一：
          "stock"                 : 公司發行之股票及表明股票權利憑證（成交價格×0.003）
          "bond"                  : 公司債及其他經政府核准之有價證券（成交價格×0.001）
          "warrant"               : 認購(售)權證（成交價格×0.001）
          "warrant_delivery_stock": 權證股票交割（履約價格×股數×0.003）
          "warrant_delivery_cash" : 權證現金結算（履約價格×股數×0.001）
      - tp: 成交價格（數值）
      - ep: 履約價格（僅對 warrant_delivery_stock 與 warrant_delivery_cash 有效）
      - sc: 股數（僅對 warrant_delivery_stock 與 warrant_delivery_cash 有效）

    回傳：以「整數+小數點兩位」格式的字串，例如 "300.25"，
           此處若 SMT 計算結果為 1916.129，則只顯示 1916.12（即截斷小數點後第三位）。
    """
    transaction_price = RealVal(tp)
    s = Solver()
    s.add(transaction_price == tp)
    
    if tax_item == "stock":
        # 稅率 0.003
        tax_expr = transaction_price * 0.003
    elif tax_item == "bond":
        # 稅率 0.001
        tax_expr = transaction_price * 0.001
    elif tax_item == "warrant":
        # 稅率 0.001
        tax_expr = transaction_price * 0.001
    elif tax_item == "warrant_delivery_stock":
        if ep is None or sc is None:
            raise ValueError("計算權證股票交割必須提供履約價格與股數")
        exercise_price = RealVal(ep)
        share_count = RealVal(sc)
        s.add(exercise_price == ep, share_count == sc)
        tax_expr = exercise_price * share_count * 0.003
    elif tax_item == "warrant_delivery_cash":
        if ep is None or sc is None:
            raise ValueError("計算權證現金結算必須提供履約價格與股數")
        exercise_price = RealVal(ep)
        share_count = RealVal(sc)
        s.add(exercise_price == ep, share_count == sc)
        tax_expr = exercise_price * share_count * 0.001
    else:
        raise ValueError("未知的證券交易稅項目")
    
    if s.check() == sat:
        m = s.model()
        tax_val = m.evaluate(tax_expr, model_completion=True)
        # 轉成 float 並截斷到小數點後兩位 (非四捨五入)
        val = z3_to_float(tax_val)
        truncated = int(val * 100) / 100
        return f"{truncated:.2f}"
    else:
        return None

def calculate_futures_tax_item(tax_item, ca, pa=None):
    """
    計算期貨交易稅中的某一項目。

    參數：
      - tax_item: 必須為下列其中之一：
          "stock_index"      : 股價類期貨（契約金額×0.00002）
          "interest_rate_30" : 30天期商業本票利率期貨（契約金額×0.000000125）
          "interest_rate_10" : 10年期政府債期貨（契約金額×0.00000125）
          "option"           : 選擇權及期貨選擇權（權利金金額×0.001）
          "gold"             : 黃金期貨（契約金額×0.0000025）
      - ca: 契約金額（數值，除 "option" 外皆以此計算）
      - pa: 權利金金額（僅對 "option" 有效）

    回傳：以「整數+小數點兩位」格式的字串，例如 "100.25"，
           同樣會截斷超過小數點後第二位的數字。
    """
    contract_amount = RealVal(ca)
    s = Solver()
    s.add(contract_amount == ca)
    
    if tax_item == "stock_index":
        tax_expr = contract_amount * 0.00002
    elif tax_item == "interest_rate_30":
        tax_expr = contract_amount * 0.000000125
    elif tax_item == "interest_rate_10":
        tax_expr = contract_amount * 0.00000125
    elif tax_item == "option":
        if pa is None:
            raise ValueError("計算選擇權稅必須提供權利金金額")
        premium_amount = RealVal(pa)
        s.add(premium_amount == pa)
        tax_expr = premium_amount * 0.001
    elif tax_item == "gold":
        tax_expr = contract_amount * 0.0000025
    else:
        raise ValueError("未知的期貨交易稅項目")
    
    if s.check() == sat:
        m = s.model()
        tax_val = m.evaluate(tax_expr, model_completion=True)
        val = z3_to_float(tax_val)
        truncated = int(val * 100) / 100
        return f"{truncated:.2f}"
    else:
        return None

# 示範如何呼叫：
if __name__ == '__main__':
    # 證券交易稅測試範例
    sec_result = calculate_securities_tax_item("stock", 100000)
    print("證券交易稅_股票：", sec_result)
    
    sec_result2 = calculate_securities_tax_item("warrant_delivery_stock", 100000, ep=50, sc=1000)
    print("證券交易稅_權證股票交割：", sec_result2)
    
    # 期貨交易稅測試範例
    fut_result = calculate_futures_tax_item("option", 5000000, pa=20000)
    print("期貨交易稅_選擇權及期貨選擇權：", fut_result)


證券交易稅_股票： 300.00
證券交易稅_權證股票交割： 150.00
期貨交易稅_選擇權及期貨選擇權： 20.00


In [2]:
#############################
# 隨機測試參數產生並顯示結果
#############################
import random 

def generate_securities_tests():
    print("===== 證券交易稅 隨機 5 組 測試參數及結果 =====")
    securities_options = ["stock", "bond", "warrant", "warrant_delivery_stock", "warrant_delivery_cash"]
    # 成交價格 (tp) 範圍
    tp_range = (1000, 1000000)
    # 履約價格 (ep) 範圍 (僅適用於 warrant_delivery_*)
    ep_range = (10, 100)
    # 股數 (sc) 範圍 (僅適用於 warrant_delivery_*)
    sc_range = (100, 10000)
    
    for i in range(5):
        tax_item = random.choice(securities_options)
        if tax_item in ["stock", "bond", "warrant"]:
            tp = random.randint(tp_range[0], tp_range[1])
            result = calculate_securities_tax_item(tax_item, tp)
            print(f"Test {i+1:03d}: {tax_item}, tp = {tp}, expected tax = {result}")
        else:
            tp = random.randint(tp_range[0], tp_range[1])
            ep = random.randint(ep_range[0], ep_range[1])
            sc = random.randint(sc_range[0], sc_range[1])
            result = calculate_securities_tax_item(tax_item, tp, ep, sc)
            print(f"Test {i+1:03d}: {tax_item}, tp = {tp}, ep = {ep}, sc = {sc}, expected tax = {result}")

def generate_futures_tests():
    print("\n===== 期貨交易稅 隨機 5組 測試參數及結果 =====")
    futures_options = ["stock_index", "interest_rate_30", "interest_rate_10", "option", "gold"]
    # 契約金額 (ca) 範圍
    ca_range = (1000000, 50000000)
    # 權利金 (pa) 範圍 (僅對 "option" 有效)
    pa_range = (10000, 200000)
    
    for i in range(5):
        tax_item = random.choice(futures_options)
        if tax_item == "option":
            ca = random.randint(ca_range[0], ca_range[1])
            pa = random.randint(pa_range[0], pa_range[1])
            result = calculate_futures_tax_item(tax_item, ca, pa)
            print(f"Test {i+1:03d}: {tax_item}, ca = {ca}, pa = {pa}, expected tax = {result}")
        else:
            ca = random.randint(ca_range[0], ca_range[1])
            result = calculate_futures_tax_item(tax_item, ca)
            print(f"Test {i+1:03d}: {tax_item}, ca = {ca}, expected tax = {result}")

if __name__ == '__main__':
    generate_securities_tests()
    generate_futures_tests()


===== 證券交易稅 隨機 5 組 測試參數及結果 =====
Test 001: warrant_delivery_cash, tp = 504592, ep = 80, sc = 3014, expected tax = 241.12
Test 002: warrant_delivery_cash, tp = 348959, ep = 23, sc = 1385, expected tax = 31.85
Test 003: bond, tp = 94728, expected tax = 94.72
Test 004: warrant_delivery_cash, tp = 69724, ep = 23, sc = 2419, expected tax = 55.63
Test 005: bond, tp = 274702, expected tax = 274.70

===== 期貨交易稅 隨機 5組 測試參數及結果 =====
Test 001: stock_index, ca = 43499903, expected tax = 869.99
Test 002: interest_rate_10, ca = 28518272, expected tax = 35.64
Test 003: stock_index, ca = 39484868, expected tax = 789.69
Test 004: stock_index, ca = 29593040, expected tax = 591.86
Test 005: interest_rate_30, ca = 7165961, expected tax = 0.89
